I want to read all spells and create a table with all relevant information

# setup

In [1]:
import os
import re
import pandas as pd

from tqdm import tqdm

# utils

In [2]:
def parse_argument(arg):
    arg = arg.strip()
    if not arg:
        return None
    if arg == 'true':
        return True
    elif arg == 'false':
        return False
    elif arg == 'nil':
        return None
    if (arg.startswith('"') and arg.endswith('"')) or (arg.startswith("'") and arg.endswith("'")):
        return arg[1:-1]
    else:
        try:
            return eval(arg)
        except NameError:
            return arg
        except:
            return arg

In [3]:
def split_arguments(s):
    args = []
    current = []
    in_quote = None
    escape = False
    for c in s:
        if escape:
            current.append(c)
            escape = False
        elif c == '\\':
            escape = True
        elif in_quote:
            if c == in_quote:
                in_quote = None
            current.append(c)
        elif c in ('"', "'"):
            in_quote = c
            current.append(c)
        elif c == ',' and not in_quote:
            args.append(''.join(current).strip())
            current = []
        else:
            current.append(c)
    if current:
        args.append(''.join(current).strip())
    return args

In [4]:
def parse_spell_file(file_path):
    spell_dict = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.split('--')[0].strip()
            if line.startswith('spell:'):
                match = re.match(r'spell:(\w+)\((.*)\)', line)
                if match:
                    method = match.group(1)
                    params_str = match.group(2)
                    args = split_arguments(params_str)
                    parsed_args = [parse_argument(arg) for arg in args if arg]
                    if parsed_args:
                        spell_dict[method] = parsed_args[0] if len(parsed_args) == 1 else parsed_args
    return spell_dict

# main

In [5]:
root_dir = '.'

In [6]:
spells = []
for root, _, files in tqdm(os.walk(root_dir)):
    for file in files:
        if file.endswith('.lua'):
            file_path = os.path.join(root, file)
            spell_dict = parse_spell_file(file_path)
            if spell_dict:
                spell_dict['path'] = file_path
                spells.append(spell_dict)

8it [00:00, 83.36it/s]


In [7]:
file, spell_dict

('ultimate_light.lua',
 {'name': 'Ultimate Light',
  'words': 'utevo vis lux',
  'group': 'support',
  'vocation': ['druid;true',
   'elder druid;true',
   'sorcerer;true',
   'master sorcerer;true'],
  'castSound': 'SOUND_EFFECT_TYPE_SPELL_ULTIMATE_LIGHT',
  'id': 75,
  'cooldown': 2000,
  'groupCooldown': 2000,
  'level': 26,
  'mana': 140,
  'isSelfTarget': True,
  'isAggressive': False,
  'isPremium': True,
  'needLearn': False,
  'path': '.\\support\\ultimate_light.lua'})

In [8]:
df = pd.DataFrame(spells)
len(df), len(df.columns), df.columns

(173,
 31,
 Index(['name', 'runeId', 'id', 'level', 'magicLevel', 'needTarget',
        'isAggressive', 'allowFarUse', 'charges', 'vocation', 'path', 'group',
        'words', 'castSound', 'impactSound', 'mana', 'isPremium', 'range',
        'blockWalls', 'needWeapon', 'cooldown', 'groupCooldown', 'needLearn',
        'needCasterTargetOrDirection', 'needDirection', 'isSelfTarget', 'soul',
        'hasParams', 'hasPlayerNameParam', 'allowOnSelf', 'isBlockingWalls'],
       dtype='object'))

In [9]:
df.to_csv('spells.csv')
df

,name,runeId,id,level,magicLevel,needTarget,isAggressive,allowFarUse,charges,vocation,...,groupCooldown,needLearn,needCasterTargetOrDirection,needDirection,isSelfTarget,soul,hasParams,hasPlayerNameParam,allowOnSelf,isBlockingWalls
0,test rune,3162.0,220,20.0,5.0,True,False,True,25.0,"[sorcerer;true, master sorcerer;true]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Annihilation,NaN,62,110.0,NaN,True,NaN,NaN,NaN,"[knight;true, elite knight;true]",...,2000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apprentice's Strike,NaN,169,8.0,NaN,NaN,NaN,NaN,NaN,"[druid;true, elder druid;true, sorcerer;true, ...",...,2000,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Berserk,NaN,80,35.0,NaN,NaN,NaN,NaN,NaN,"[knight;true, elite knight;true]",...,2000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brutal Strike,NaN,61,16.0,NaN,True,NaN,NaN,NaN,"[knight;true, elite knight;true]",...,2000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Sharpshooter,NaN,135,60.0,NaN,NaN,False,NaN,NaN,"[paladin;true, royal paladin;true]",...,"[2000, 10000]",False,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
169,Strong Haste,NaN,39,20.0,NaN,NaN,False,NaN,NaN,"[druid;true, elder druid;true, sorcerer;true, ...",...,2000,False,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
170,Summon Creature,NaN,9,25.0,NaN,NaN,NaN,NaN,NaN,"[druid;true, sorcerer;true, elder druid;true, ...",...,2000,False,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
171,Swift Foot,NaN,134,55.0,NaN,NaN,False,NaN,NaN,"[paladin;true, royal paladin;true]",...,"[2000, 10000]",False,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
